## Produce daily Origin-Destination trip matrix for NYC resident cbgs

##### Can aggregate to different NYC geo summary levels and destination geo summary levels, separates visits to home cbg
##### Output is daily csv o-d matrices

In [ ]:
import pandas as pd
import numpy as np
import s3fs
import os
import time

In [ ]:
from geo import stco,sub
from safegraph_py_functions import safegraph_py_functions as sgpy

In [ ]:
%load_ext dotenv
%dotenv
myAccessKey = os.getenv('myAccessKey')
mySecretKey = os.getenv('mySecretKey')

start = time.time()

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# read nyc origin cbgs
cbg_nyc = pd.read_csv(f'../data/nyc_cbg.csv')

In [ ]:
# specify the SG key and secret
fs = s3fs.S3FileSystem(profile="safegraphws", key=myAccessKey, secret=mySecretKey, client_kwargs={'endpoint_url': 'https://s3.wasabisys.com', 'region_name':'us-east-1'})

In [ ]:
# set date variables, in this case, a test month for 2 different years
month = "04"
years = ["2019","2020"]
dayList =["01","02","03","04","05","06","07","08","09","10","11","12","13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31"]

# If running all days and months - replace range and adjust for loop to run through lists below:
#monthList =["01","02","03","04","05","06","07","08"]
#dayNumList =[31, 29, 31, 30, 31, 30, 31,31] 

In [ ]:
## Test month - individual days for year-over-year comparison

#frames = [] \can add if using loop
for y in years:
    for i in range(0,30):
        with fs.open(f'sg-c19-response/social-distancing/v2/{y}/{month}/{dayList[i]}/{y}-{month}-{dayList[i]}-social-distancing.csv.gz','rb') as f:
            print(f'{y}-{month}-{dayList[i]}')
            # read SG's file
            df = pd.read_csv(f, escapechar='\\', compression='gzip')
            # filter NYC's Origin CBGs
            df = pd.merge(cbg_nyc, df, left_on="orig_cbg", right_on="origin_census_block_group", how="inner")

            #unpack json destination cbgs
            df = sgpy.unpack_json_and_merge(df, json_column='destination_cbgs', key_col_name='destination_cbg', value_col_name='dest_cbg_count')

            ##Make new columns
            df['orig_cbg'] = df['orig_cbg'].apply(str) #clean origin cbg
            #separate home trips from other trips for later aggregation
            df['is_home'] = df.apply(lambda x: x['orig_cbg']==x['destination_cbg'],axis=1)
            #id destinations by county fips
            df['dest_stco_all'] = df['destination_cbg'].str[:5]
            #id 31cr counties and all others outside
            df['dest_stco_reg'] = df['dest_stco_all'] 
            df.loc[~df['dest_stco_reg'].isin(stco),'dest_stco_reg'] = 'O31CR' 
            df['dest_sub'] = df['dest_stco_all'].map(sub).fillna('O31CR')

            #Make new table with select columns
            dff = df[['orig_stco','dest_cbg_count','dest_stco_reg','dest_sub','is_home']]
            dff = dff.groupby(['dest_stco_reg','dest_sub','orig_stco','is_home']).agg({'dest_cbg_count':np.sum}).reset_index()
            dff = pd.pivot_table(dff,values=['dest_cbg_count'],index='dest_stco_reg',columns=['orig_stco','is_home'],aggfunc=np.sum,fill_value=0)

            dff.to_csv(f'output/{month}_test/{y}-{month}{dayList[i]}.csv')
            
            #mid = time.time()
            #elapsed_mid = mid - start
            #print(f'Mid run time - {elapsed_mid}')

In [ ]:
end = time.time()
elapsed = end - start
print(f'Run time - {elapsed} seconds')